### Load tensorflow

In [0]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd

### Check all columns in the dataset

In [11]:
data = pd.read_csv('prices.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Drop columns `date` and  `symbol`

In [0]:
data_1 = data.drop(['date','symbol'],axis=1)

In [18]:
data_1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [62]:
#data_2 = data_1.head(1000)
data_2.shape

(1000, 5)

### Convert Float64 to Float32

In [0]:
data_3 = data_2.astype(dtype='float32')

In [24]:
data_3.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Divide the data into train and test sets

In [0]:

from sklearn.model_selection import train_test_split

In [0]:
X=data_3.drop('close',axis=1)
y=data_3['close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [34]:
X_train.shape

(800, 4)

### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
train_x = norm.fit_transform(X_train)

In [0]:
import numpy as np
train_y = np.array(y_train)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [66]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(train_x, train_y, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(train_y, prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 3334.4077
Current Loss on iteration 1 3334.4084
Current Loss on iteration 2 3334.4075
Current Loss on iteration 3 3334.4075
Current Loss on iteration 4 3334.4067
Current Loss on iteration 5 3334.4077
Current Loss on iteration 6 3334.4077
Current Loss on iteration 7 3334.4072
Current Loss on iteration 8 3334.4077
Current Loss on iteration 9 3334.4072
Current Loss on iteration 10 3334.4072
Current Loss on iteration 11 3334.4072
Current Loss on iteration 12 3334.408
Current Loss on iteration 13 3334.408
Current Loss on iteration 14 3334.4077
Current Loss on iteration 15 3334.4075
Current Loss on iteration 16 3334.4072
Current Loss on iteration 17 3334.4065
Current Loss on iteration 18 3334.4072
Current Loss on iteration 19 3334.4077
Current Loss on iteration 20 3334.4087
Current Loss on iteration 21 3334.4072
Current Loss on iteration 22 3334.4072
Current Loss on iteration 23 3334.4072
Current Loss on iteration 24 3334.4065
Current Loss on iteration 25 3334.406

### Get the shapes and values of W and b

In [56]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5492334e-03]
 [2.5264884e-03]
 [2.5697988e-03]
 [3.2567165e+01]]
Bias:
 [32.567173]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.models.Sequential()

#Comile the model
model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])

### Execute the model

In [60]:
model.fit(train_x, train_y, epochs=5)

Epoch 1/5
800/800 [==============================] - 0s 262us/sample - loss: 7684.8513 - acc: 0.0000e+00
Epoch 2/5
800/800 [==============================] - 0s 29us/sample - loss: 7684.8512 - acc: 0.0000e+00
Epoch 3/5
800/800 [==============================] - 0s 29us/sample - loss: 7684.8512 - acc: 0.0000e+00
Epoch 4/5
800/800 [==============================] - 0s 29us/sample - loss: 7684.8512 - acc: 0.0000e+00
Epoch 5/5
800/800 [==============================] - 0s 26us/sample - loss: 7684.8514 - acc: 0.0000e+00


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
import pandas as pd
iris = pd.read_csv('Iris.csv')

### Splitting the data into feature set and target set

In [73]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
features = iris.drop(columns= ['Species']) 

target = iris[['Species']]

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
Y_processed = pd.DataFrame(target).apply(LabelEncoder().fit_transform)

In [0]:
testY = tf.keras.utils.to_categorical(Y_processed, num_classes=3)

In [79]:
testY.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split
transformer = Normalizer()
#features = transformer.fit_transform(features)

#train_x, test_x,train_y,test_y = train_test_split(features, testY, test_size=.3, random_state=7)

In [0]:
train_x, test_x,train_y,test_y = train_test_split(features, testY, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [87]:
model.fit(train_x.values, train_y, validation_split=0.2, epochs=30)

Train on 84 samples, validate on 21 samples
Epoch 1/30
84/84 [==============================] - 0s 137us/sample - loss: 0.3685 - acc: 0.8929 - val_loss: 0.6230 - val_acc: 0.6190
Epoch 2/30
84/84 [==============================] - 0s 161us/sample - loss: 0.3786 - acc: 0.8333 - val_loss: 0.6132 - val_acc: 0.6190
Epoch 3/30
84/84 [==============================] - 0s 124us/sample - loss: 0.3555 - acc: 0.8810 - val_loss: 0.5979 - val_acc: 0.6190
Epoch 4/30
84/84 [==============================] - 0s 113us/sample - loss: 0.3458 - acc: 0.8690 - val_loss: 0.5857 - val_acc: 0.6190
Epoch 5/30
84/84 [==============================] - 0s 123us/sample - loss: 0.3774 - acc: 0.8810 - val_loss: 0.5674 - val_acc: 0.6190
Epoch 6/30
84/84 [==============================] - 0s 114us/sample - loss: 0.3337 - acc: 0.8929 - val_loss: 0.5562 - val_acc: 0.6190
Epoch 7/30
84/84 [==============================] - 0s 122us/sample - loss: 0.3235 - acc: 0.8810 - val_loss: 0.5466 - val_acc: 0.6190
Epoch 8/30
84/84 [

### Report Accuracy of the predicted values

In [88]:
model.evaluate(test_x,test_y)

45/45 [==============================] - 0s 2ms/sample - loss: 0.3577 - acc: 0.8000


[0.35766193999184503, 0.8]